In [13]:
import pandas as pd
import numpy as np
import fnmatch
import os
import re
import nltk
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing

from nltk.corpus import stopwords
stopwords = set(stopwords.words("english"))

class classifier_workshop():
    
    def get_files(self, tem_path):
        data = pd.DataFrame()
        for root, dir, files in os.walk(tem_path):
            for items in fnmatch.filter(files, "*"):
                tmp = root.split("/")
                tmp = tmp[len(tmp)-1]
                if ".git" not in tmp:
                    abs_path = tem_path+tmp+"/"+items
                    data = data.append({"path":abs_path, "label":tmp}, ignore_index=True)
        return data
    
    def clean_text(self, df):
        the_new_df = pd.DataFrame()
        for file_name, the_label in zip(df["path"][1:], df["label"][1:]):
        #     print file_name
        #     print the_label
            f = open(file_name, "r")
            tem_text = f.readlines()
            tem_text = " ".join(tem_text)
            the_text = tem_text.split()
            the_text = [word.lower() for word in the_text]
            the_text = [re.sub(r'[^a-z]', '', taken) for taken in the_text]
            the_text = [word for word in the_text if word != ""]
            the_text = [word for word in the_text if word not in stopwords]
            the_text = " ".join(the_text)

            the_new_df = the_new_df.append({"body":the_text, "label":the_label}, ignore_index=True)
            f.close()
        return the_new_df
    
    def train_model(self, model, df):
        vectorizer = TfidfVectorizer(max_features = 1000, ngram_range = (1,3))
        the_vec = vectorizer.fit_transform(df.body).toarray()
        tfidf = pd.DataFrame(the_vec)
        tfidf.columns = vectorizer.get_feature_names()  

        label_enc = preprocessing.LabelEncoder()
        label_enc.fit(df[df.columns[1]])

        tfidf[df.columns[1]] = [label_enc.transform([word])[0] for
                                            word in df[df.columns[1]]]
        the_score = cross_val_score(model, tfidf[tfidf.columns[0:-1]], 
                        tfidf[tfidf.columns[-1]], cv = 18)

        model.fit(tfidf[tfidf.columns[0:-1]], tfidf[tfidf.columns[-1]])
        return vectorizer, model, label_enc

    def the_classify(self, v, m, l, to_predict = ["six"]):
        tfidf_pred = v.transform(to_predict)

        prediction = m.predict(tfidf_pred)
        the_prob = pd.DataFrame(m.predict_proba(tfidf_pred))
        the_prob.columns = l.classes_
        the_pred = l.inverse_transform([prediction][0])

        return the_pred
        

In [2]:
the_path = "/Users/tianchenwang/workshop_data/"

In [3]:
the_framework = classifier_workshop()

the_df = the_framework.get_files(the_path)

In [4]:
the_new_df = the_framework.clean_text(the_df)

In [5]:
v, m, l = the_framework.train_model(RandomForestClassifier(), the_new_df)


/Users/tianchenwang/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [14]:
the_framework.the_classify(v, m, l, ['six as ten oneaea adf'])

array(['hiking'], dtype=object)

In [ ]:
def train_model(model, df):
        vectorizer = TfidfVectorizer(max_features = 1000, ngram_range = (1,3))
        the_vec = vectorizer.fit_transform(df.body).toarray()
        tfidf = pd.DataFrame(the_vec)
        tfidf.columns = vectorizer.get_feature_names()  

        label_enc = preprocessing.LabelEncoder()
        label_enc.fit(df[df.columns[1]])

        tfidf[df.columns[1]] = [label_enc.transform([word])[0] for
                                            word in df[df.columns[1]]]
        the_score = cross_val_score(model, tfidf[tfidf.columns[0:-1]], 
                        tfidf[tfidf.columns[-1]], cv = 18)

        model.fit(tfidf[tfidf.columns[0:-1]], tfidf[tfidf.columns[-1]])
        return vectorizer, model, label_enc